In [9]:
import ujson
import pprint
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from operator import itemgetter
import multiprocessing
from copy import deepcopy

from itertools import ifilter


pp = pprint.PrettyPrinter(indent=4)
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Users/pmulrooney/anaconda2/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [10]:
def log_progress(sequence, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')

In [14]:
with open("/Users/pmulrooney/class/notebooks/capstone/ReferenceEndomondoWorkouts/pmulrooney_run.json", 'r') as f:
    endo = ujson.load(f)

In [15]:
for _i in sorted(endo.keys()):
    if type(endo[_i]) not in (dict, list):
        print _i, ':', endo[_i]
    else:
        print _i, ':', type(endo[_i])

admin_rejected : False
altitude_max : 6.6
altitude_min : -9.7
ascent : 0.0
author : <type 'dict'>
calories : 1231.6
can_copy : True
descent : 10.0
distance : 16.0960006714
duration : 4844.59
expand : full
feed_id : 281475874731619
hashtags : <type 'list'>
hydration : 0.89
id : 871971752
include_in_stats : True
is_live : False
is_peptalk_allowed : False
laps : <type 'dict'>
local_start_time : 2017-02-12T14:54:49.000-05:00
pb_count : 8
personal_bests : <type 'list'>
pictures : <type 'list'>
points : <type 'dict'>
records : <type 'list'>
show_map : 0
show_workout : 0
small_encoded_polyline : ch~fEpaijUDiAfDkAxArFdD`l@pNz{ClD|ToBxHoBIcAoE_MdCqIpP_GvD{ExM{RzY|ApDgBnAcEzMpGjFtGmC|KhEhKNxKcDdIiGgAmHyFwA\uCnCEzDvDjApPYrLwBeAyHtG_TTy@[CsLeGm@uE{CyEvCuBQ_EqD^oEgBgDn\uh@tE}L|CGxK{Q@qBtLaB`AjE`CCbAy@T}G_CwJi@mWsO}{Bt@wG{Cwm@m@wGsA{Cw@t@
speed_avg : 11.96088883
speed_max : 16.7816
sport : 0
start_time : 2017-02-12T19:54:49.000Z
tagged_users : <type 'list'>
weather : <type 'dict'>


In [16]:
cum_dist = 0.0
duration = 0.0
for _i in endo['points']['points']:
    cum_dist += _i['distance']
    duration += _i['duration']


In [17]:
print cum_dist
print duration

4069.987
1217768000.0


In [18]:
for _i in endo['points']['points']:
    print _i
    break

{u'distance': 0.0, u'sensor_data': {}, u'instruction': 2, u'longitude': -117.1972076, u'duration': 0.0, u'time': u'2017-02-12T19:54:49.000Z', u'latitude': 32.7643452}


In [19]:
endo['points']['points'][199:202]

[{u'altitude': -3.7,
  u'distance': 6.564,
  u'duration': 1953000.0,
  u'latitude': 32.767169,
  u'longitude': -117.2497195,
  u'sensor_data': {u'speed': 13.0414},
  u'speed': 13.0414,
  u'time': u'2017-02-12T20:27:22.000Z'},
 {u'altitude': -3.6,
  u'distance': 6.583,
  u'duration': 1959000.0,
  u'latitude': 32.7670118,
  u'longitude': -117.2496973,
  u'sensor_data': {u'speed': 10.9308},
  u'speed': 10.9308,
  u'time': u'2017-02-12T20:27:28.000Z'},
 {u'altitude': -3.6,
  u'distance': 6.611,
  u'duration': 1967000.0,
  u'latitude': 32.7668032,
  u'longitude': -117.2498716,
  u'sensor_data': {u'speed': 11.0243},
  u'speed': 11.0243,
  u'time': u'2017-02-12T20:27:36.000Z'}]

In [20]:
import math

math.sqrt((32.7668032 - 32.7670118)**2 + (-117.2498716+117.2496973)**2)

0.0002718353361952054

In [21]:
endo['points']['points'][:3]

[{u'distance': 0.0,
  u'duration': 0.0,
  u'instruction': 2,
  u'latitude': 32.7643452,
  u'longitude': -117.1972076,
  u'sensor_data': {},
  u'time': u'2017-02-12T19:54:49.000Z'},
 {u'altitude': 2.0,
  u'distance': 0.019,
  u'duration': 21000.0,
  u'latitude': 32.7643371,
  u'longitude': -117.1970196,
  u'sensor_data': {u'speed': 6.7593},
  u'speed': 6.7593,
  u'time': u'2017-02-12T19:55:10.000Z'},
 {u'altitude': 1.9,
  u'distance': 0.037,
  u'duration': 27000.0,
  u'latitude': 32.7643108,
  u'longitude': -117.1968344,
  u'sensor_data': {u'speed': 11.3106},
  u'speed': 11.3106,
  u'time': u'2017-02-12T19:55:16.000Z'}]

In [22]:
len(endo['points']['points'])

500

In [30]:
_prev = (endo['points']['points'][0]['latitude'], endo['points']['points'][0]['longitude'])

total_distance = 0.0

for _point in endo['points']['points'][1:]:
    _current = (_point['latitude'], _point['longitude'])
    _tmp_distance = math.sqrt( (_current[0] - _prev[0])**2 + (_current[1] - _prev[1])**2 )
    
    total_distance += _tmp_distance
    
    
    _prev = _current

In [31]:
total_distance

0.1617062428118401

In [46]:
print 'workoutid', 'diff_altitude', 'geo_distance'
print endo['id'], ",", endo['altitude_max'] - endo['altitude_min'],",",  total_distance
print ""
print 'heart_rate_avg', 'speed_avg', 'elapsed_time'
print "0",",",  endo['speed_avg'],",",  endo['duration']
print ""
print "User:", 212295

workoutid diff_altitude geo_distance
871971752 , 16.3 , 0.161706242812

heart_rate_avg speed_avg elapsed_time
0 , 11.96088883 , 4844.59

User: 212295


In [43]:
print "Unnamed: 0|workoutid|route_prediction|perf_prediction|  userid|     diff_altitude|  geo_distance|heart_rate_avg|    speed_avg|elapsed_time|   user_avg_speed| user_avg_dist|"

Unnamed: 0|workoutid|route_prediction|perf_prediction|  userid|     diff_altitude|  geo_distance|heart_rate_avg|    speed_avg|elapsed_time|   user_avg_speed| user_avg_dist|


In [50]:
print "Unnamed: 0,workoutid,route_prediction,perf_prediction,userid,diff_altitude,geo_distance,heart_rate_avg,speed_avg,elapsed_time,user_avg_speed,user_avg_dist"
print "%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s"%("0", endo['id'], "??", "??", "212295", endo['altitude_max'] - endo['altitude_min'], total_distance, "0", endo['speed_avg'], endo['duration'], '12.7922508', '4845.92074')

Unnamed: 0,workoutid,route_prediction,perf_prediction,userid,diff_altitude,geo_distance,heart_rate_avg,speed_avg,elapsed_time,user_avg_speed,user_avg_dist
0,871971752,??,??,212295,16.3,0.161706242812,0,11.96088883,4844.59,12.7922508,4845.92074


In [ ]:
+Unnamed          0 
+workoutid        871971752
+route_prediction ??
+perf_prediction  ??
+userid           212295
+diff_altitude    16.3
+geo_distance     0.161706242812
+heart_rate_avg   0
+speed_avg        11.96088883
+elapsed_time     4844.59
+user_avg_speed   10.0258328805181818
+user_avg_dist    0.14012889348074766355